# CRAWL DỮ LIỆU SOUNDCLOUD VỚI API

## BẢNG PHÂN CÔNG CÔNG VIỆC

STT&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Thông tin sinh viên&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Mức độ hoàn thành&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Công việc
<br>
1&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&ensp;&nbsp;Nguyễn Đăng Thi-19127276&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl Api với track và playlist
<br>
2&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;Lê Minh Trí-19127592&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl data bằng Selenium với track và playlist
<br>
3&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;Phan Vĩ Giai-19127639&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl Api với users
<br>
4&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;Võ Nhất Huy-19127642&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp;&nbsp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl data bằng Selenium với user và link lấy link dẫn từ home page tới playlist 

## CÁCH CHẠY PROJECT
**Chọn Kernel -> Restart & Run All**

In [1]:
# Import necessary libary
import sys
import time
import csv
import json
import os.path
import requests
import pandas as pd
import requests_cache
import urllib.request
import multiprocessing
from threading import Thread

In [8]:
# Function to cache which are completed results
def filter_response(response):
    if '"incomplete_results":true' in response.text:
        return False
    return True

requests_cache.install_cache(
    cache_name="request-cache",
    expire_after=None,
    allowable_methods=("GET"),
    filter_fn=filter_response,
)

In [9]:
class Crawl:
    # Init class
    def __init__(self, user_file, playlist_file, track_file):
        # Time sleep interval
        self.time_sleep = 2
        # fields
        self.user_fields = [
            "id",
            "username",
            "first_name",
            "last_name",
            "full_name",
            "city",
            "avatar_url",
            "playlist_count",
            "track_count",
            "description",
            "followers_count",
            "followings_count",
            "groups_count",
            "likes_count",
            "comments_count",
            "playlist_likes_count",
            "last_modified",
            "created_at",
            "verified",
        ]
        self.playlist_fields = [
            "id",
            "user_id",
            "title",
            "created_at",
            "last_modified",
            "license",
            "likes_count",
            "permalink_url",
            "public",
            "purchase_url",
            "reposts_count",
            "secret_token",
            "sharing",
            "tag_list",
            "uri",
            "set_type",
            "is_album",
            "trackIds",
        ]
        self.track_fields = [
            "id",
            "user_id",
            "title",
            "description",
            "track_format",
            "uri",
            "caption",
            "comment_count",
            "created_at",
            "duration",
            "genre",
            "last_modified",
            "license",
            "likes_count",
            "permalink_url",
            "playback_count",
            "reposts_count",
            "state",
        ]
        # Use this prefix for writing to computer
        self.prefix = "Api_data/"
        # Client id
        self.client_id = "FjnXkiGFvyaVIYtXadMm9pqIDawoxzUW"
        # Target file
        self.user_file = user_file
        self.playlist_file = playlist_file
        self.track_file = track_file
        # Invoking create csv file
        self.init_file_csv(self.prefix + user_file, self.user_fields)
        self.init_file_csv(self.prefix + playlist_file, self.playlist_fields)
        self.init_file_csv(self.prefix + track_file, self.track_fields)
    
    # Create header for file csv
    def init_file_csv(self, file_name, fields):
        file_exists = os.path.isfile(file_name)
        with open(file_name, "a") as csvfile:
            csvwriter = csv.DictWriter(csvfile, delimiter='\t', lineterminator='\n', fieldnames=fields)
            if not file_exists:
                csvwriter.writeheader()
            
    # Write to csv file with delimiter tab
    def write_to_csv(self, file_name, contents, fields=None):
        with open(file_name, "a") as csvfile:
            csvwriter = csv.writer(csvfile, delimiter='\t') 
            csvwriter.writerows(contents)

    # Get information of json object with specified fields
    def get_info(self, response, fields):
        json = response.json()
        content = [
            "NULL" if json[f] == "" or json[f] == "null" else str(json[f])
            for f in fields
        ]
        return content

    # Parse json data for playlist unity
    def get_playlist_info(self, response, fields):
        ids = []
        json = response.json()
        content = [
            "NULL" if json[f] == "" or json[f] == "null" else str(json[f])
            for f in fields[:-1]
        ]
        ids = [str(track["id"]) for track in json["tracks"]]
        content.append(",".join(ids))
        return content

    # Get all playlist id
    def get_playlist_ids(self, user_id):
        ids = []
        fail_json = False
        user_link = f"https://api-v2.soundcloud.com/users/{user_id}/playlists?&client_id={self.client_id}"
        while True:
            # for each user id , make api call to get user resources
            response = requests.get(user_link)
            # check each failed flag may occur
            if response.ok == True:
                break
            if not response.json():
                fail_json = True
                break
            time.sleep(self.time_sleep)
        if not fail_json:
            while True:
                if '"incomplete_results":true' in response.text:
                    response = requests.get(user_link)
                else:
                    break
            json = response.json()
            # for collection field , parse playlist id then save in string type with ',' delimiter
            ids = [str(playlist["id"]) for playlist in json["collection"]]
        return ids

    # Read track_ids list of each playlist in playlist.csv
    def read_track_ids_from_playlists_csv(self):
        rows = []
        reader = csv.reader(open(f"{self.prefix}playlists.csv"), delimiter="\t")
        skip = next(reader)
        for row in reader:
            if row[-1] != "":
                rows.extend([id for id in row[-1].split(",")])
        return rows

    # Read playlist_id in users.csv
    def read_user_ids_from_users(self):
        rows = []
        csvfile = open(f"{self.prefix}users.csv")
        data = csv.reader((line.replace("\0", "") for line in csvfile), delimiter="\t")
        skip = next(data)
        for row in data:
            rows.append(int(row[0]))
        return rows

    # Main crawling all users
    def do_crawling_users(self, n_threads, start, step):
        # Start all threads.
        threads = []
        for n in range(n_threads):
            # Init thread
            t = Thread(
                target=self.crawl_user, args=(start, start + step - 1, n + 1)
            )
            start += step - 1
            t.daemon = True
            t.start()
            threads.append(t)
        # Wait all threads to finish.
        for t in threads:
            t.join()

    # Crawl one user
    def crawl_user(self, start, end, thread):
        # Store all result in contents python list
        contents = []
        for i in range(start, end):
            fail_json = False
            # Target link to get json object
            user_link = (
                f"https://api-v2.soundcloud.com/users/{i}?client_id={self.client_id}"
            )
            while True:
                # Request link to get response
                response = requests.get(user_link)
                if response.ok == True:
                    break
                # If response is failed, we turn on the flag
                if not response.json():
                    fail_json = True
                    break
                time.sleep(self.time_sleep)
            # If the flag turn on, we do not permiss to get into this function
            if not fail_json:
                while True:
                    if '"incomplete_results":true' in response.text:
                        response = requests.get(user_link)
                    else:
                        break
                # Parse response into useful information with target fields
                result = self.get_info(response, self.user_fields)
                # Append result into db
                contents.append(result)
                # To speed up process, we use batch job algorithm to save data into file (batch size: 200 records)
                if len(contents) > 200:
                    # Save batch records into file
                    self.write_to_csv(self.prefix + self.user_file, contents, self.user_fields)
                    print(
                        f"{time.ctime()} -- Running job: {Crawl.crawl_user.__name__} with thread-{thread}"
                    )
                    print(f"Saving batch job size: {len(contents)}")
                    print("-------------------------------------")
                    # Clear db
                    contents.clear()
                    time.sleep(self.time_sleep)
        # After all, save the remaining data into file
        print('Done --------------------------------')
        print(f"{time.ctime()} -- Running job: {Crawl.crawl_user.__name__} with thread-{thread}")
        print(f"Saving batch job size: {len(contents)}")
        print("-------------------------------------")
        self.write_to_csv(self.prefix + self.user_file, contents, self.user_fields)
        return

    # Crawl one playlist
    def crawl_playlist(self, user_ids, thread):
        # Store  playlist fields for each playlist data recieved from api call
        content_records = []
        for user_id in user_ids:
            # For each user read from user.csv , make api call to get all playlist_id of that user
            playlist_ids = self.get_playlist_ids(user_id)
            # For each playlist id of each user , make api call to retrieve data
            for playlist_id in playlist_ids:
                # Fpi url for playlists resources
                playlist_link = f"https://api-v2.soundcloud.com/playlists/{playlist_id}?client_id={self.client_id}"
                while True:
                    response = requests.get(playlist_link)
                    # Ff GET request is failed , continue with other other id
                    if response.ok == True:
                        break
                    time.sleep(self.time_sleep)
                while True:
                    # If request queries exceed the time limit , keep make making api call until "incomplete_results":false
                    if '"incomplete_results":true' in response.text:
                        response = requests.get(user_link)
                    else:
                        break
                # Farse response to filter needed playlists data
                content = self.get_playlist_info(response, self.playlist_fields)
                content_records.append(content)
                # Save and write data in batch to reduce executing time for file writing (.e.g save 100 record for each batch)
                if len(content_records) > 200:
                    # Save records to file with batch size
                    self.write_to_csv(self.prefix + self.playlist_file, content_records, self.playlist_fields)
                    print(
                        f"{time.ctime()} -- Running job: {Crawl.crawl_playlist.__name__} with thread-{thread}"
                    )
                    print(f"Saving batch job size: {len(content_records)}")
                    print("-------------------------------------")
                    # Clear db
                    content_records.clear()
                    time.sleep(self.time_sleep)
        # After all, save the remaining data into file
        print('Done --------------------------------')
        print(f"{time.ctime()} -- Running job: {Crawl.crawl_playlist.__name__} with thread-{thread}")
        print(f"Saving batch job size: {len(content_records)}")
        print("-------------------------------------")
        self.write_to_csv(self.prefix + self.playlist_file, content_records, self.playlist_fields)
        return

    # Main crawling all playlists
    def do_crawling_playlist(self, n_threads):
        # Read user id saved in file users.csv to get playlist resources
        user_ids = self.read_user_ids_from_users()
        # Start with index 0 , for each thread executed , number of api call(step) is (number of user id)/(number of thread)
        start = 0
        step = int(len(user_ids) / n_threads)
        playlist_threads = []
        # Split crawling job into multiple concurrent crawling process
        for n in range(n_threads):
            t = Thread(
                target=self.crawl_playlist,
                args=(user_ids[start : start + step - 1], n + 1),
            )
            start += step - 1
            t.daemon = True
            t.start()
            playlist_threads.append(t)
        # Wait all threads to finish.
        for t in playlist_threads:
            t.join()

    # Crawl one track
    def crawl_track(self, track_ids, thread):
        # Store  playlist fields for each track data recieved from api call
        content_records = []
        # For each track_id , make api to get track resources
        for track_id in track_ids:
            track_link = f"https://api-v2.soundcloud.com/tracks/{track_id}?client_id={self.client_id}"
            while True:
                response = requests.get(track_link)
                # If GET request is failed , continue with other other id
                if response.ok == True:
                    break
                # Wait for api call
                time.sleep(self.time_sleep)
            while True:
                # If request queries exceed the time limit , keep make making api call until "incomplete_results":false
                if '"incomplete_results":true' in response.text:
                    response = requests.get(user_link)
                else:
                    break
            # Parse response to filter needed tracks data
            content = self.get_info(response, self.track_fields)
            content_records.append(content)
            # Save and write tracks data in batch for decreseing file writing executing time
            if len(content_records) > 200:
                # Save records to file with batch size
                self.write_to_csv(self.prefix + self.track_file, content_records, self.track_fields)
                print(f"{time.ctime()} -- Running job: {Crawl.crawl_track.__name__} with thread-{thread}")
                print(f"Saving batch job size: {len(content_records)}")
                print("-------------------------------------")
                # Clear db
                content_records.clear()
                time.sleep(self.time_sleep)
        # After all, save the remaining data into file
        print('Done --------------------------------')
        print(f"{time.ctime()} -- Running job: {Crawl.crawl_track.__name__} with thread-{thread}")
        print(f"Saving batch job size: {len(content_records)}")
        print("-------------------------------------")
        self.write_to_csv(self.prefix + self.track_file, content_records, self.track_fields)
        return

    # Main crawling all tracks
    def do_crawling_track(self, n_threads):
        # Read track id saved in file playlist.csv to get track resources
        track_ids = self.read_track_ids_from_playlists_csv()
        # Start with index 0 , for each thread executed , number of api call(step) is (number of track id)/(number of thread)
        start = 0
        step = int(len(track_ids) / n_threads)
        track_threads = []
        # Split crawling job into multiple concurrent crawling process
        for n in range(n_threads):
            t = Thread(
                target=self.crawl_track,
                args=(track_ids[start : start + step - 1], n + 1),
            )
            start += step - 1
            t.daemon = True
            t.start()
            track_threads.append(t)
        # Wait all threads to finish.
        for t in track_threads:
            t.join()

In [ ]:
%%time
if __name__ == "__main__":
    # Target file name to save users, playlists and tracks
    user_file_name = "users.csv"
    playlist_file_name = "playlists.csv"
    track_file_name = "tracks.csv"
    
    # Initialize Class Object
    crawl_obj = Crawl(user_file_name, playlist_file_name, track_file_name)

    # Main running with 7 threads for each job to get high performance
    crawl_obj.do_crawling_users(10, 1, 200)
    crawl_obj.do_crawling_playlist(10)
    crawl_obj.do_crawling_track(10)

# **Crawl user**

**Ý tưởng thực hiện**
<br>

- Sau một hồi đọc document trên mạng, search tìm thông tin trên stackoverflow, thì ta cũng tìm được link API để lấy thông tin user như sau:
<br>

https://api-v2.soundcloud.com/users/552289491?client_id=Your_Client_id
<br>

- Parameter mình quan tâm nhất đó chính là **User_id**
<br>

**Phương pháp thực hiện việc crawling users**
<br>

**A. Cách tiếp cận đầu tiên**
- Ban đầu thì loay hoay tìm **User_id** bằng cách sẽ kết hợp với Parse HTML để có thể lấy được **User_id**
- Cách này thì quá tốn công, cũng như phải viết code nhiều hơn.

**B. Cách tiếp cận hoàn hảo**
- Sau một vài tiếng thì mới nhớ lại những kiến thức đã học ở môn ***Database*** hồi kì trước, ta có một cách tiêp cận hoàn hảo hơn phần trên.
- Suy luận User_id là một ***Surrogate Key*** - một loại khóa chỉ có mục đích chính là tăng dần và định danh dữ liệu. Ta test thử ***User_id*** '*sơ khai đầu tiên'*:
<br>

https://api-v2.soundcloud.com/users/1?client_id=Your_Client_id
<br>

- Woww!! Suy luận của mình đã đúng.

### **Kết luận**
Vậy phương pháp cuối cùng của chúng ta là dùng một vòng for chạy từ 1 -> n (chúng ta tự giả sử SoundCloud có bao nhiêu users)

In [2]:
# Sau khi crawl xong thông tin users, minh xem thông tin 5 users đầu
df = pd.read_csv("Api_data/users.csv", sep="\t")
df.head()

,id,username,first_name,last_name,full_name,city,avatar_url,playlist_count,track_count,description,followers_count,followings_count,groups_count,likes_count,comments_count,playlist_likes_count,last_modified,created_at,verified
0,1195,johnvon,john,von,john von,Los Angeles,https://i1.sndcdn.com/avatars-000072165579-wn8...,6,17,"Jon Hassell band, A Thousand Miles from Nowher...",775,221,0,89,22,8,2020-08-02T17:07:36Z,2008-01-18T21:53:00Z,False
1,1196,lupo69,NaN,NaN,NaN,None,https://a1.sndcdn.com/images/default_avatar_la...,0,0,None,10,1,0,0,0,0,2009-03-05T16:58:32Z,2008-01-19T12:20:56Z,False
2,1198,florianf,Florian,Filsinger,Florian Filsinger,None,https://a1.sndcdn.com/images/default_avatar_la...,0,0,None,12,2,0,0,0,0,2009-03-05T16:58:32Z,2008-01-19T17:30:58Z,False
3,1199,numblog,Matthias,Gutjahr,Matthias Gutjahr,Wiesbaden,https://i1.sndcdn.com/avatars-000000009604-d86...,0,0,"In my New Urban Music Blog, I write about Brok...",491,353,0,235,56,3,2016-02-01T13:40:14Z,2008-01-19T20:16:00Z,False
4,1200,Sharon Safav,Sharon,Safav,Sharon Safav,Beer-Sheva,https://i1.sndcdn.com/avatars-000055753273-2e8...,1,6,NaN,183,23,0,49,25,2,2017-11-04T08:15:23Z,2008-01-20T01:16:19Z,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1763 entries, 0 to 1762
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    1763 non-null   int64 
 1   username              1763 non-null   object
 2   first_name            1421 non-null   object
 3   last_name             1271 non-null   object
 4   full_name             1429 non-null   object
 5   city                  1617 non-null   object
 6   avatar_url            1763 non-null   object
 7   playlist_count        1763 non-null   int64 
 8   track_count           1763 non-null   int64 
 9   description           1466 non-null   object
 10  followers_count       1763 non-null   int64 
 11  followings_count      1763 non-null   int64 
 12  groups_count          1763 non-null   int64 
 13  likes_count           1763 non-null   int64 
 14  comments_count        1763 non-null   int64 
 15  playlist_likes_count  1763 non-null   

### **Kết quả đạt được**

**A. Nhận xét**
<br>
- Với cách áp dụng phương pháp trên, ta thực nghiệm cho n = 2000 (có nghĩa sẽ lấy từ 1 -> 2000 users)
- Nhưng kết quả cho ta thấy, có nhiềù **Users_id** mà không có ai (Có thể do người dùng xóa tài khoản, gian lận,...)

**B. Kết quả**
<br>
- Ta chỉ lấy những trường dữ liệu mà ta quan tâm gồm **19 trường**.
- Tổng số users thu thập được từ **SoundCloud** là: **1763 users.**

**C. Giải thích**
<br> 
Kết quả trường dữ liệu ta thu thập được gồm 19 trường với ý nghĩa sau:
- **id**: Định danh cho từng user
- **username**: Tên của user đặt trong hệ thống
- **first_name**: Họ của user
- **last_name**: Tên của user
- **full_name**: Tên đầy đủ của user
- **city**: Thành phố đang sống
- **avatar_url**: Địa chỉ đến avatar của user
- **playlist_count**: Số playlists mà user đang sỡ hữu
- **track_count**: Số tracks mà user đang sỡ hữu
- **description**: Mô tả bản thân, ...
- **followers_count**: Sô người follow user
- **followings_count**: Số người user đang follow
- **groups_count**: Số group đang tham gia
- **likes_count**: Số yêu thích bài hát
- **comments_count**: Sô bình luận
- **playlist_likes_count**: Số yêu thích playlist
- **last_modified**: Thời gian cho biết những thay đổi đến tài khỏan user
- **created_at**: Thời gian tạo tài khoản
- **verified**: Tài khoản có được xác thực hay chưa ?

# **Crawl playlist**

**Ý tưởng thực hiện**
<br>
- Sau khi tìm được danh sách user_id , việc lấy dữ liệu trở nên đơn giản hơn:
<br>
- Để tìm danh sách playlist , khi đã có được user_id , ta gọi API call với path:
<br>

https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id=Your_Client_id
<br>
- Sau khi gọi API ta sẽ thu được danh sách những playlist_id của user đó
<br>

**Phương pháp thực hiện việc crawling playlist**
<br>

**Cách tiếp cận**
- Từ danh sách user_id đã lấy được , ta sẽ trích ra danh sách user_id 
- Gọi API theo path đã đề cập ở trên
- Xử lý response để thu về các thông tin cần thiết
- Đặt biệt , trong phần data thu về, có phần thông tin về ID của từng track của trong playlist đó , ta lại xử lý để biến đổi về cách chuôi String cách nhau bởi dấu phây để tiện cho việc xử lý với track
<br>



In [4]:
# Sau khi crawl xong thông tin playlists, minh xem thông tin 5 playlists đầu
df_playlist = pd.read_csv("Api_data/playlists.csv", sep="\t")
df_playlist.head()

,id,user_id,title,created_at,last_modified,license,likes_count,permalink_url,public,purchase_url,reposts_count,secret_token,sharing,tag_list,uri,set_type,is_album,trackIds
0,472348,767,Robert Nesta - «The Son of Sun Mix - 1»,2010-12-13T18:54:43Z,2010-12-13T18:55:12Z,all-rights-reserved,0,https://soundcloud.com/robertnesta/sets/robert...,True,None,0,None,public,deephouse,https://api.soundcloud.com/playlists/472348,NaN,False,8014824
1,472146,767,Robert Nesta - «No House Mix»,2010-12-13T16:50:05Z,2010-12-13T16:50:06Z,all-rights-reserved,0,https://soundcloud.com/robertnesta/sets/robert...,True,None,0,None,public,Nujazz acidjazz hiphop reggae downbeat chillout,https://api.soundcloud.com/playlists/472146,NaN,False,NaN
2,803896233,770,Live @ Blue Marlin Ibiza - June 2019,2019-06-14T12:16:32Z,2019-06-14T13:11:42Z,all-rights-reserved,6,https://soundcloud.com/djremo/sets/live-blue-m...,True,None,1,None,public,NaN,https://api.soundcloud.com/playlists/803896233,NaN,False,"636556752,636571083"
3,38424366,770,Remo - Scammers E.P.,2014-06-06T14:51:12Z,2015-10-13T14:53:44Z,all-rights-reserved,11,https://soundcloud.com/djremo/sets/remo-scammers,True,http://www.beatport.com/release/scammers/1323426,2,None,public,remo dj scammers ep novotek tech house second ...,https://api.soundcloud.com/playlists/38424366,ep,True,NaN
4,19343766,770,Remo presents : S3V3N,2014-01-07T15:53:16Z,2014-05-26T15:28:28Z,all-rights-reserved,8,https://soundcloud.com/djremo/sets/s3v3n,True,None,3,None,public,"House ""Tech House"" remo podcast seven s3v3n",https://api.soundcloud.com/playlists/19343766,NaN,False,NaN


In [5]:
df_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2175 entries, 0 to 2174
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2175 non-null   int64 
 1   user_id        2175 non-null   int64 
 2   title          2175 non-null   object
 3   created_at     2175 non-null   object
 4   last_modified  2175 non-null   object
 5   license        2175 non-null   object
 6   likes_count    2175 non-null   int64 
 7   permalink_url  2175 non-null   object
 8   public         2175 non-null   bool  
 9   purchase_url   2149 non-null   object
 10  reposts_count  2175 non-null   int64 
 11  secret_token   2175 non-null   object
 12  sharing        2175 non-null   object
 13  tag_list       822 non-null    object
 14  uri            2175 non-null   object
 15  set_type       709 non-null    object
 16  is_album       2175 non-null   bool  
 17  trackIds       2003 non-null   object
dtypes: bool(2), int64(4), object

### **Kết quả đạt được**

**A. Nhận xét**
<br>
- Với từng user , mỗi người chỉ có thể có nhiều playlist , những cách playlist có thể rỗng , nên sẽ cần lượt bỏ trong lúc thu thập

**B. Kết quả**
<br>
- Ta chỉ lấy những trường dữ liệu mà ta quan tâm gồm **18 trường**.
- Tổng số playlist thu được: **2175 playlists** 

**C. Giải thích**
<br> 
Kết quả trường dữ liệu ta thu thập được gồm 18 trường với ý nghĩa sau:
- **id**: Định danh cho từng user
- **user_id**: ID của user tạo playlist
- **title** : Tên của playlist
- **created_at** : Thời điểm playlist được tạo
- **last_modified** : Thời điểm playlist được sửa đổi lần cuối
- **license** :Loại bản bản của được đăng của playlist
- **likes_count** : Số lượng like của playlists
- **permalink_url** : Link dẫn đến playlist
- **public** : Playlist có thuộc tính public không?
- **purchase_url** :Link dẫn đến trang để mua playlist
- **reposts_count** : Số lượng playlist được share ở nơi khác
- **secret_token** : tokens với để truy cập playlist nếu là public
- **sharing** :Chế độ chia sẻ của playlist
- **tag_list**:Thẻ tag playlist
- **uri**: URI của playlist
- **set_type**: Loại playlist
- **is_album** : Có phải playlist thuộc album
- **trackIds**: danh sách các track id của playlist

# **Crawl track**

**Ý tưởng thực hiện**
<br>
- Tương tự như playlist, sau khi tìm được danh sách playlist_id , việc lấy dữ liệu của track có thể đươc nhờ trường **Ids_string** đã được lưu ở lần lấy của playlist:
<br>
- Sau khi có track id , ta gọi API:
<br>

https://api-v2.soundcloud.com/tracks/{track_id}/?client_id=Your_Client_id
<br>
- Sau khi gọi API ta sẽ thu được thông tin của track đó cùng với user đã upload 
<br>

**Phương pháp thực hiện việc crawling tracks**
<br>

**Cách tiếp cận**
- Từ danh sách playlist_id đã lấy được , ta sẽ trích ra danh sách track qua trường IDs gồm cách track_id cách nhau bởi ','
- Gọi API theo path đã đề cập ở trên
- Xử lý response để thu về cách thông tin cần thiết
<br>

In [6]:
# Sau khi crawl xong thông tin tracks, minh xem thông tin 5 tracks đầu
df_track = pd.read_csv("Api_data/tracks.csv", sep="\t")
df_track.head()

,id,user_id,title,description,track_format,uri,caption,comment_count,created_at,duration,genre,last_modified,license,likes_count,permalink_url,playback_count,reposts_count,state
0,1066418635,69,Markus Enochson - Spaced Out,None,single-track,https://api.soundcloud.com/tracks/1066418635,None,0,2021-06-11T10:50:38Z,30000,Electronic,2021-06-11T11:18:22Z,all-rights-reserved,0,https://soundcloud.com/markusenochson/markus-e...,3,0,finished
1,1066436512,69,Markus Enochson - You Got It (w/ Cornelia),None,single-track,https://api.soundcloud.com/tracks/1066436512,None,0,2021-06-11T11:08:07Z,30000,Electronic,2021-06-11T11:32:42Z,all-rights-reserved,0,https://soundcloud.com/markusenochson/markus-e...,3,0,finished
2,1066424482,69,Markus Enochson feat. Masaya - No Stoppin This,None,single-track,https://api.soundcloud.com/tracks/1066424482,None,0,2021-06-11T10:55:40Z,30000,Electronic,2021-06-11T11:20:06Z,all-rights-reserved,0,https://soundcloud.com/markusenochson/markus-e...,5,0,finished
3,1066434958,69,Markus Enochson - Tonight The Night (w/ Jocely...,None,single-track,https://api.soundcloud.com/tracks/1066434958,None,0,2021-06-11T11:05:19Z,30000,Electronic,2021-06-11T11:33:49Z,all-rights-reserved,0,https://soundcloud.com/markusenochson/markus-e...,1,0,finished
4,1066404655,69,Markus Enochson - Take Me Away (w/ Cornelia),None,single-track,https://api.soundcloud.com/tracks/1066404655,None,0,2021-06-11T10:38:51Z,30000,Electronic,2021-06-11T10:52:41Z,all-rights-reserved,0,https://soundcloud.com/markusenochson/markus-e...,3,0,finished


In [7]:
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10854 entries, 0 to 10853
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              10854 non-null  int64 
 1   user_id         10854 non-null  int64 
 2   title           10854 non-null  object
 3   description     8914 non-null   object
 4   track_format    10854 non-null  object
 5   uri             10854 non-null  object
 6   caption         10854 non-null  object
 7   comment_count   10854 non-null  object
 8   created_at      10854 non-null  object
 9   duration        10854 non-null  int64 
 10  genre           9394 non-null   object
 11  last_modified   10854 non-null  object
 12  license         10854 non-null  object
 13  likes_count     10854 non-null  object
 14  permalink_url   10854 non-null  object
 15  playback_count  10854 non-null  object
 16  reposts_count   10854 non-null  int64 
 17  state           10854 non-null  object
dtypes: int

### **Kết quả đạt được**

**A. Kết quả**
<br>
- Ta chỉ lấy những trường dữ liệu mà ta quan tâm gồm **18 trường**.
- Tổng số tracks thu được: **10854 tracks**

**B. Giải thích**
<br> 
Kết quả trường dữ liệu ta thu thập được gồm 18 trường với ý nghĩa sau:
- **id** : ID của track
- **user_id** : ID của user upload track
- **title** :Tiêu đề của track
- **description** : Mô tả của track
- **track_format** : Thông tin phân loại của track
- **uri** : URI của track
- **caption**
- **comment_count** : Số lượng comment dành cho track
- **created_at** : Thởi điểm tạo track
- **duration** : Độ dài của track
- **genre** : Thể loại của track
- **last_modified** : Thời điểm sửa đổi lần cuối
- **license** : bản của track
- **likes_count** :Số lượng like của track
- **permalink_url** : Link dẫn đến track
- **playback_count**: Tổng của số phát của track
- **reposts_count** : Sô lần track được chia sẻ ở nơi khác
- **state** :Trạng thái của track